# Задание 1

- ### *Вывести всех студентов из определённой группы (запросить номер группы у пользователя)*

Импорт необходимых пакетов

In [2]:
import os
import pymysql
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd

Ищем файл .env с переменными окружения и загружаем их

In [ ]:
load_dotenv()

Подключаемся к БД функцией connect, указав параметры подключения из переменных окружения

In [ ]:
connection = pymysql.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME"),
    cursorclass=pymysql.cursors.DictCursor,
)

connection

Переключим активную БД командой USE, с которой будут выполняться все дальнейшие SQL-операции

In [ ]:
cursor = connection.cursor()
cursor.execute("USE railway;")

Создадим датафрейм из CSV-таблицы

In [6]:
df_students = pd.read_csv("input/students.csv")

Также преобразуем колонку birthday в формат datetime

In [7]:
df_students["birthdate"] = pd.to_datetime(df_students["birthdate"])

In [ ]:
df_students.info()

In [ ]:
df_students.head()

Создадим объект engine

In [10]:
engine = create_engine(
    f"mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/railway"
)

Запишем данные из датафрейма в таблицу SQL-базы данных

In [ ]:
df_students.to_sql("students", engine, if_exists="replace", index=False)

Убедимся что данные были успешно добавлены в таблицу

In [ ]:
cursor.execute(
    """
    SELECT * FROM students LIMIT 10;
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

Преобразуем в самой таблице БД колонку birthdate из формата DATETIME в DATE

In [13]:
cursor.execute(
    """
    UPDATE students
    SET birthdate = DATE(birthdate);
    """
)

connection.commit()

cursor.execute(
    """
    ALTER TABLE students
    MODIFY birthdate DATE;
    """
)

connection.commit()

В SQL запросе метода execute укажем номер группы, полученную через функцию input

In [ ]:
group = input("Введите номер группы, чтобы получить список студентов: ")

cursor.execute(
    f"""
    SELECT *
    FROM students
    WHERE group_number = {group};
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

- ### *Вывести студентов, родившихся после 2003 года*

Необходимо отфильтровать таблицу по колонке даты рождения. Для этого в SQL запросе метода execute просто укажем условие - дату рождения

In [ ]:
birthdate_after = "2003-12-31"

cursor.execute(
    f"""
    SELECT *
    FROM students
    WHERE birthdate > '{birthdate_after}';
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

- ### *Вывести средний балл по всей таблице*

Используя средства SQL, применим функцию AVG с указанием необходимой колонки

In [ ]:
cursor.execute(
    """
    SELECT AVG(score)
    FROM students;
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

- ### *Вывести топ-3 студентов по баллу (DESC)*

Для сортировки выполняют выборку и применяют оператор ORDER BY с указаниекм колонки. Параметры DESC и ASC влияют на порядок сортировки. Для получения только 3-х студентов, используем параметр LIMIT для ограничения количества записей

In [ ]:
cursor.execute(
    """
    SELECT *
    FROM students
    ORDER BY score DESC
    LIMIT 3;
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

- ### *Вывести количество студентов в каждой группе (GROUP BY)*

Группируем все строки таблицы по столбцу group_number и для каждой такой группы с помощью COUNT считаем количество строк в ней, присваивая этому числу псевдоним

In [ ]:
cursor.execute(
    """
    SELECT group_number, COUNT(*) AS cnt
    FROM students
    GROUP BY group_number;
    """
)

results = cursor.fetchall()

for row in results:
    print(row)

connection.commit()

Закрытие соединения и очистка ресурсов

In [ ]:
cursor.close()
connection.close()
engine.dispose()